<a href="https://colab.research.google.com/github/Squirrelcoding/parking-meter/blob/main/Cool_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip3 install OSMPythonTools
!pip3 install gdal

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.7 MB/s eta 0:00:00
  Created wheel for OSMPythonTools: filename=OSMPythonTools-0.3.5-py3-none-any.whl size=32716 sha256=b2d91855f86c8281e08e8048e86a1ed53898e22f86fa3e66d3151c5b4ba57b70
  Stored in directory: /root/.cache/pip/wheels/c6/d0/f3/df5ca1a29d749e1a99e635f84617944cfc2d0fa2f5d8078208
Successfully built OSMPythonTools


In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='poopnet-4fb22')

In [ ]:
# TODO: Write some code that gets all of the boundaries of a parking lot given its ID or something
from OSMPythonTools.api import Api
api = Api()

In [ ]:
import os
import geemap
from osgeo import gdal
from PIL import Image

def tif_to_png(filename: str | os.PathLike):
  pre = filename.split(".")[0]
  print(pre)
  tif = gdal.Open(filename)
  arr = tif.ReadAsArray().transpose(1, 2, 0)  # Convert CHW to HWC
  img = Image.fromarray(arr.astype(np.uint8))
  os.remove(filename)
  img.save(pre + '.png')

def download_parking_lot(way_id: int):
  way = api.query(f"way/{way_id}")
  roi = ee.Geometry.Polygon(way.geometry()['coordinates'])
  image_collection = (
      ee.ImageCollection('USDA/NAIP/DOQQ')
      .filterBounds(roi)  # Filter to images intersecting the ROI
      .filterDate('2022-01-01', '2023-12-31') # Filter to a recent time frame
      .sort('system:time_start', False) # Sort by date, newest first
  )

  mosaic = image_collection.mosaic()

  vis_params = {
      'bands': ['R', 'G', 'B'],
      'min': 0,
      'max': 255
  }

  clipped_image = mosaic.clip(roi).unmask()
  geemap.ee_export_image(clipped_image, filename=f'{way_id}.tif', scale=0.10394100844860077, region=roi)

In [ ]:
m = geemap.Map()

way = api.query(f"way/{id}")
roi = ee.Geometry.Polygon(way.geometry()['coordinates'])
image_collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(roi)  # Filter to images intersecting the ROI
    .filterDate('2022-01-01', '2023-12-31') # Filter to a recent time frame
    .sort('system:time_start', False) # Sort by date, newest first
)

mosaic = image_collection.mosaic()

vis_params = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255
}
clipped_image = mosaic.clip(roi).unmask()
m.center_object(clipped_image)
m.add_layer(clipped_image, vis_params, "")

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Getting the most common color of the parking lot

In [ ]:
from OSMPythonTools.overpass import Overpass

overpass = Overpass()

first_coordinate = (44.886320, -93.328578)  # (lat, lon)
second_coordinate = (44.86709, -93.30564)   # (lat, lon)

south = min(first_coordinate[0], second_coordinate[0])
north = max(first_coordinate[0], second_coordinate[0])
west = min(first_coordinate[1], second_coordinate[1])
east = max(first_coordinate[1], second_coordinate[1])

query = f"""
(
  way["amenity"="parking"]({south},{west},{north},{east});
  relation["amenity"="parking"]({south},{west},{north},{east});
);
out body;
>;
out skel qt;
"""

result = overpass.query(query)

In [ ]:
def get_dominant_color(pil_img):
    img = pil_img.copy()
    img = img.convert("RGBA")
    img = img.resize((1, 1), resample=0)
    dominant_color = img.getpixel((0, 0))
    return dominant_color

In [ ]:
n = 30
sum_colors = np.array([0.0, 0.0, 0.0])

colors = []
for parking_lot in result.toJSON()['elements'][0:n]:
  id = parking_lot['id']
  download_parking_lot(id)
  print(id)
  try:
    tif_to_png(f"{id}.tif")
  except AttributeError:
    print("File too big. Skipping.")
    continue
  img = Image.open(f"{id}.png")
  color = np.array(get_dominant_color(img)[0:3]).astype(np.float32)
  sum_colors += color
  colors.append(color)
  os.remove(f"{id}.png")

sum_colors / n

In [ ]:
colors